# Q1

In [18]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data/results-gpt4o.csv")

In [4]:
df = df.iloc[:300]

In [5]:
df.head()

,answer_llm,answer_orig,document,question,course
0,The CONTEXT provided does not give specific in...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up for the Machine Learning Zoomc...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,"No, the provided context does not include any ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [6]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-mpnet-base-dot-v1"


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
embedding_model = SentenceTransformer(model_name)

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [8]:
answer_llm = df.iloc[0].answer_llm

In [10]:
emdedding = embedding_model.encode(answer_llm)

In [12]:
emdedding[0]

-0.23882452

# Q2

In [13]:
def compute_dot_product(embedding1, embedding2):
    return np.dot(embedding1, embedding2)

In [19]:
evaluations = []
for idx, row in df.iterrows():
    emb_llm = embedding_model.encode(row['answer_llm'])
    emb_orig = embedding_model.encode(row['answer_orig'])
    score = compute_dot_product(emb_llm, emb_orig)
    evaluations.append(score)

In [20]:
percentile_75 = np.percentile(evaluations, 75)

print("75th Percentile of Scores:", percentile_75)

75th Percentile of Scores: 32.09545612335205


# Q3

In [21]:
def normalize_vector(v):
    norm = np.linalg.norm(v)
    return v / norm

In [22]:
def compute_cosine_similarity(embedding1, embedding2):
    embedding1_norm = normalize_vector(embedding1)
    embedding2_norm = normalize_vector(embedding2)
    return np.dot(embedding1_norm, embedding2_norm)

In [24]:
evaluations = []
for idx, row in df.iterrows():
    emb_llm = embedding_model.encode(row['answer_llm'])
    emb_orig = embedding_model.encode(row['answer_orig'])
    score = compute_cosine_similarity(emb_llm, emb_orig)
    evaluations.append(score)


In [25]:
percentile_75 = np.percentile(evaluations, 75)

print("75th Percentile of Scores:", percentile_75)

75th Percentile of Scores: 0.8348871618509293
